I started by dissecting the starter notebook, and I immediately saw some red flags.

Using a simple linear model for what I knew was a non-linear problem, the naive fillna(0), and the critical error of randomly splitting time-series data were all things I flagged to fix.

My initial thinking on feature engineering was to use a 10s moving average, but after talking to Gemini, I realized a longer window (like 600s) would be better to capture the signal through the noise. 

The more professional approach I landed on was to engineer multiple windows (10s, 60s, 600s) and feed them all to the model.

The real challenge began when I tried to process the full dataset. 

The kernel crashed repeatedly under the memory pressure, even when I tried to be clever with float32 types (which caused overflows) or iterative processing. This was a classic data pipeline bottleneck. 

It forced a strategic pivot away from trying to hold one giant table in memory.But this whole thing hit a big roadblock when I scored -0.00170611 T_T

The solution was to stop thinking about merging raw data and start thinking about merging features. I built a pipeline to process each asset file independently, calculate a rich set of features (returns, realized volatility, spread, imbalance), and save the results to a lightweight cache. 

This finally solved the memory issue and let the real analysis begin.

With the data accessible, EDA uncovered two crucial, opposing facts:

Strong Autocorrelation: ETH's own implied volatility was highly persistent. Its past was a very strong predictor of its future.

Critical Anomaly: ETH's price returns showed almost zero correlation with the rest of the market, including BTC. This was a huge surprise and suggested a major data quality issue that made direct cross-asset prediction unreliable.

This forced my final, most important strategic pivot. Instead of using other assets to predict ETH directly, 

I decided to use them to create "market context" features.

Aggregated the volatility and spread from all other assets into market-wide indicators.I ended the day by building the final data pipeline and creating a validation plot that proved the hypothesis. 

The plot clearly showed ETH's implied volatility spiking dramatically at the exact same time the overall market entered a high-stress state.

The data part is now done. I have a robust, feature-rich, and validated dataset ready for modeling.

Used LightBGM and 
After hitting a solid 0.65 Pearson correlation, the next phase was a deep dive into iterative feature engineering to push the score higher. I experimented with more complex features, like volatility ratios and advanced order book metrics, but found they didn't provide a significant lift. This process was crucial as it proved that our initial, simpler feature set was the most robust and signal-rich. With the champion features locked in, I used Optuna to hyperparameter-tune the LightGBM model, squeezing out the maximum performance. This disciplined approach of feature validation followed by tuning resulted in our final, reliable out-of-fold correlation of 0.733 and a complete end-to-end pipeline.



In [2]:
# =============================================================================
#
# DEEP HOLDOUT VALIDATION SCRIPT
#
# =============================================================================

# =============================================================================
# 1. IMPORTS & SETUP (Same as before)
# =============================================================================
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import gc

print("--- Setup Complete ---")

# --- Configuration ---
DATA_DIR = Path("/kaggle/input/gq-implied-volatility-forecasting")
OUTPUT_DIR = Path("/kaggle/working/")


# =============================================================================
# 2. CHAMPION FEATURE & DATA PREPARATION (Same as before)
# =============================================================================

def calculate_champion_features(df: pd.DataFrame) -> pd.DataFrame:
    # This function is identical to the winning version from Phase 2
    df = df.sort_index()
    price_cols = ['mid_price', 'ask_price1', 'bid_price1']
    for col in price_cols:
        if col in df.columns:
            df.loc[:, col] = df[col].replace(0, np.nan)
    df['wap'] = (df['bid_price1'] * df['ask_volume1'] + df['ask_price1'] * df['bid_volume1']) / (df['bid_volume1'] + df['ask_volume1'])
    df['log_return_wap_1s'] = np.log(df['wap']).diff(1)
    weights = [0.4, 0.3, 0.15, 0.1, 0.05]
    df['weighted_bid_vol'] = (weights * df[[f'bid_volume{i}' for i in range(1, 6)]]).sum(axis=1)
    df['weighted_ask_vol'] = (weights * df[[f'ask_volume{i}' for i in range(1, 6)]]).sum(axis=1)
    df['depth_weighted_imbalance'] = (df['weighted_bid_vol'] - df['weighted_ask_vol']) / (df['weighted_bid_vol'] + df['weighted_ask_vol'])
    df['log_return_1s'] = np.log(df['mid_price']).diff(1)
    for n in [10, 30, 60, 600]:
        df[f'realized_vol_{n}s'] = df['log_return_wap_1s'].rolling(window=n).std()
    df['wap_trend_300s'] = df['wap'].diff(periods=300)
    df['spread'] = df['ask_price1'] - df['bid_price1']
    bid_vols = df[[f'bid_volume{i}' for i in range(1, 6)]].sum(axis=1)
    ask_vols = df[[f'ask_volume{i}' for i in range(1, 6)]].sum(axis=1)
    df['book_imbalance'] = (bid_vols - ask_vols) / (bid_vols + ask_vols)
    df['vol_of_vol_10s_w60'] = df['realized_vol_10s'].rolling(window=60).std()
    df['imbalance_momentum_5s'] = df['book_imbalance'].diff(periods=5)
    feature_cols = [
        'log_return_1s', 'log_return_wap_1s', 'realized_vol_10s', 'realized_vol_30s',
        'realized_vol_60s', 'realized_vol_600s', 'wap_trend_300s', 'spread',
        'book_imbalance', 'depth_weighted_imbalance', 'vol_of_vol_10s_w60', 'imbalance_momentum_5s'
    ]
    return df[feature_cols].copy()

print("✅ Phase 1 Setup Complete")

2025-08-15 03:55:31.653217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755230131.877539      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755230131.939551      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--- Setup Complete ---
✅ Phase 1 Setup Complete


In [3]:
def create_full_feature_set(data_dir):
    # This is a simplified version of the data prep function for the train set
    print("--- Processing Training Data ---")
    # (The logic here is the same as your Phase 1 data prep)
    # ... condensed for brevity but the logic is identical ...
    data_type = "train"
    all_asset_paths = glob.glob(str(data_dir / f"{data_type}/*.csv"))
    all_possible_cols = ['timestamp', 'mid_price'] + [f'{side}_{level}{i}' for side in ['bid', 'ask'] for level in ['price', 'volume'] for i in range(1, 6)] + ['label']
    eth_path = [p for p in all_asset_paths if 'ETH' in p][0]
    with open(eth_path, 'r') as f: header = f.readline().strip().split(',')
    cols_to_load = [col for col in all_possible_cols if col in header]
    df_eth_raw = pd.read_csv(eth_path, usecols=cols_to_load)
    df_eth_raw['timestamp'] = pd.to_datetime(df_eth_raw['timestamp'], format='mixed', utc=True)
    df_eth_raw = df_eth_raw.set_index('timestamp').sort_index()
    df_eth_raw = df_eth_raw[~df_eth_raw.index.duplicated(keep='first')]
    eth_features = calculate_champion_features(df_eth_raw)
    eth_df = df_eth_raw[['label']].join(eth_features)
    cross_asset_features_list = []
    for asset_path in all_asset_paths:
        if 'ETH' in asset_path or 'submission' in asset_path: continue
        with open(asset_path, 'r') as f: header = f.readline().strip().split(',')
        cols_to_load = [col for col in all_possible_cols if col in header]
        df_asset_raw = pd.read_csv(asset_path, usecols=cols_to_load)
        df_asset_raw['timestamp'] = pd.to_datetime(df_asset_raw['timestamp'], format='mixed', utc=True)
        df_asset_raw = df_asset_raw.set_index('timestamp').sort_index()
        df_asset_raw = df_asset_raw[~df_asset_raw.index.duplicated(keep='first')]
        market_features = calculate_champion_features(df_asset_raw)
        cross_asset_features_list.append(market_features)
    market_df = pd.concat(cross_asset_features_list, axis=1)
    market_context_df = pd.DataFrame(index=market_df.index)
    market_context_df['market_realized_vol_60s'] = market_df.filter(like='realized_vol_60s').mean(axis=1)
    market_context_df['market_depth_weighted_imbalance'] = market_df.filter(like='depth_weighted_imbalance').mean(axis=1)
    final_df = eth_df.join(market_context_df, how='left')
    final_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    final_df = final_df.ffill().fillna(0)
    print(f"✅ Final feature set created. Shape: {final_df.shape}")
    return final_df

train_df = create_full_feature_set(DATA_DIR)
X = train_df.drop('label', axis=1)
y = train_df['label']

--- Processing Training Data ---


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(uf

✅ Final feature set created. Shape: (631292, 15)


In [4]:
# =============================================================================
# 3. DEEP HOLDOUT SPLIT
# =============================================================================
print("\n--- Creating Deep Holdout Split ---")
N_SPLITS = 5
tscv = TimeSeriesSplit(n_splits=N_SPLITS)
all_splits = list(tscv.split(X))

# We train on the first N-1 folds and hold out the very last one
train_indices = np.concatenate([all_splits[i][0] for i in range(N_SPLITS - 1)])
holdout_indices = all_splits[N_SPLITS - 1][1]

X_train, X_holdout = X.iloc[train_indices], X.iloc[holdout_indices]
y_train, y_holdout = y.iloc[train_indices], y.iloc[holdout_indices]

print(f"Training set size: {len(X_train)}")
print(f"Holdout set size: {len(X_holdout)}")


--- Creating Deep Holdout Split ---
Training set size: 1052158
Holdout set size: 105215


In [5]:
# =============================================================================
# 4. TRAIN THE ENSEMBLE ON FOLDS 0-3
# =============================================================================
# --- Scale features ---
feature_scaler = MinMaxScaler()
X_train_scaled = feature_scaler.fit_transform(X_train)

# --- Train LGBM Specialist ---
print("\n--- Training Specialist LGBM on Folds 0-3... ---")
lgbm_params = {
    'learning_rate': 0.03, 'num_leaves': 64, 'max_depth': 7,
    'objective': 'regression_l1', 'n_estimators': 1500, 'random_state': 42,
    'n_jobs': -1, 'colsample_bytree': 0.7, 'subsample': 0.8
}
lgbm_model = lgb.LGBMRegressor(**lgbm_params)
lgbm_model.fit(X_train_scaled, y_train)

# --- Train LSTM Corrector ---
print("--- Training Corrector LSTM on in-sample residuals... ---")
lgbm_train_preds = lgbm_model.predict(X_train_scaled)
train_residuals = y_train - lgbm_train_preds

residual_scaler = MinMaxScaler(feature_range=(-1, 1))
train_residuals_scaled = residual_scaler.fit_transform(train_residuals.values.reshape(-1, 1))

def create_error_sequences(data, look_back=50):
    X_seq, y_seq = [], []
    for i in range(len(data) - look_back):
        X_seq.append(data[i:(i + look_back), 0])
        y_seq.append(data[i + look_back, 0])
    return np.array(X_seq), np.array(y_seq)

LOOK_BACK = 50
X_residuals, y_residuals = create_error_sequences(train_residuals_scaled, LOOK_BACK)
X_residuals = np.reshape(X_residuals, (X_residuals.shape[0], X_residuals.shape[1], 1))

lstm_corrector = Sequential([
    LSTM(units=30, return_sequences=True, input_shape=(LOOK_BACK, 1)), Dropout(0.2),
    LSTM(units=30), Dropout(0.2), Dense(units=1)
])
lstm_corrector.compile(optimizer='adam', loss='mean_squared_error')
lstm_corrector.fit(X_residuals, y_residuals, epochs=20, batch_size=1024, verbose=1)



--- Training Specialist LGBM on Folds 0-3... ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.065366 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3086
[LightGBM] [Info] Number of data points in the train set: 1052158, number of used features: 13
[LightGBM] [Info] Start training from score 0.000040
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
--- Training Corrector LSTM on in-sample residuals... ---


I0000 00:00:1755230282.398813      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


I0000 00:00:1755230287.720964     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


1028/1028 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 0.0194
Epoch 2/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0039
Epoch 3/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0019
Epoch 4/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 9.6345e-04
Epoch 5/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 6.0747e-04
Epoch 6/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 5.3314e-04
Epoch 7/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 5.0192e-04
Epoch 8/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 4.7707e-04
Epoch 9/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 4.6213e-04
Epoch 10/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 4.3961e-04
Epoch 11/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 4.2913e-04
Epoch 12/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 4.2442e-04
Epoch 13/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 4.0864e-04
Epoch 14/20
1028/1028 ━━━━━━━━━━━━━━━━━━━━ 1

In [6]:
# =============================================================================
# 5. DIAGNOSTIC RUN ON A SMALL SAMPLE
# =============================================================================
print("\n--- Running a short diagnostic on the holdout set... ---")

# --- Predict with LGBM (this is fast) ---
X_holdout_scaled = feature_scaler.transform(X_holdout)
holdout_lgbm_preds = lgbm_model.predict(X_holdout_scaled)

# --- Predict corrections with LSTM ---
initial_sequence = train_residuals_scaled[-LOOK_BACK:]
current_sequence = initial_sequence.reshape(1, LOOK_BACK, 1)
holdout_corrections_scaled = []

# +++ DIAGNOSTIC CHANGE HERE +++
# We will only run for 500 steps to measure the speed
num_diagnostic_steps = 500
print(f"  - Generating {num_diagnostic_steps} rolling predictions from LSTM to diagnose speed...")

import time
start_time = time.time()

for i in range(num_diagnostic_steps):
    pred = lstm_corrector.predict(current_sequence, verbose=0)
    holdout_corrections_scaled.append(pred[0, 0])
    current_sequence = np.append(current_sequence[:, 1:, :], pred.reshape(1, 1, 1), axis=1)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n--- Diagnostic Complete ---")
print(f"Time to perform {num_diagnostic_steps} predictions: {elapsed_time:.2f} seconds.")

# --- Extrapolate the full runtime ---
time_per_step = elapsed_time / num_diagnostic_steps
estimated_full_runtime_seconds = time_per_step * len(X_holdout)
estimated_full_runtime_minutes = estimated_full_runtime_seconds / 60

print(f"Average time per prediction: {time_per_step*1000:.2f} milliseconds.")
print(f"Estimated runtime for the FULL holdout set: {estimated_full_runtime_minutes:.1f} minutes.")


--- Running a short diagnostic on the holdout set... ---
  - Generating 500 rolling predictions from LSTM to diagnose speed...

--- Diagnostic Complete ---
Time to perform 500 predictions: 34.29 seconds.
Average time per prediction: 68.58 milliseconds.
Estimated runtime for the FULL holdout set: 120.3 minutes.


In [ ]:
# =============================================================================
# 5. PREDICT & EVALUATE ON HOLDOUT SET (FOLD 4) - CORRECTED
# =============================================================================
print("\n--- Evaluating pipeline on the Deep Holdout Set... ---")

# --- Predict with LGBM ---
print("  - Scaling holdout features...")
X_holdout_scaled = feature_scaler.transform(X_holdout)
print(f"    Holdout scaled features shape: {X_holdout_scaled.shape}")

print("  - Predicting with LGBM...")
holdout_lgbm_preds = lgbm_model.predict(X_holdout_scaled)
print(f"    LGBM predictions shape: {holdout_lgbm_preds.shape}")
print(f"    Sample LGBM preds: {holdout_lgbm_preds[:5]}")

# --- Predict corrections with LSTM ---
print("  - Preparing initial LSTM sequence from train residuals...")
initial_sequence = train_residuals_scaled[-LOOK_BACK:]
print(f"    Initial sequence shape: {initial_sequence.shape}")
print(f"    Initial sequence sample: {initial_sequence[:5].flatten()}")

current_sequence = initial_sequence.reshape(1, LOOK_BACK, 1)
holdout_corrections_scaled = []

print("  - Generating rolling predictions from LSTM...")
for i in range(len(X_holdout_scaled)):
    pred = lstm_corrector.predict(current_sequence, verbose=0)
    holdout_corrections_scaled.append(pred[0, 0])

    if i < 3:  # Print only first few steps
        print(f"    Step {i+1} | LSTM pred (scaled): {pred[0,0]:.6f} | Current seq shape: {current_sequence.shape}")
    
    # +++ CORRECTED LINE +++
    current_sequence = np.append(current_sequence[:, 1:, :], pred.reshape(1, 1, 1), axis=1)

holdout_corrections_scaled = np.array(holdout_corrections_scaled)
print(f"  - LSTM scaled corrections shape: {holdout_corrections_scaled.shape}")
print(f"    Sample scaled corrections: {holdout_corrections_scaled[:5]}")

# Inverse transform corrections
holdout_corrections = residual_scaler.inverse_transform(
    holdout_corrections_scaled.reshape(-1, 1)
).flatten()
print(f"  - LSTM corrections (original scale) shape: {holdout_corrections.shape}")
print(f"    Sample corrections: {holdout_corrections[:5]}")

# --- Ensemble and Calculate Final Score ---
print("  - Combining LGBM predictions and LSTM corrections...")
holdout_ensemble_preds = holdout_lgbm_preds + holdout_corrections
print(f"    Ensemble predictions shape: {holdout_ensemble_preds.shape}")
print(f"    Sample ensemble preds: {holdout_ensemble_preds[:5]}")

holdout_corr, _ = pearsonr(y_holdout, holdout_ensemble_preds)

print("\n======================================================")
print(f"🏆 Final Pearson Correlation on Deep Holdout Set: {holdout_corr:.5f}")
print("======================================================")



--- Evaluating pipeline on the Deep Holdout Set... ---
  - Scaling holdout features...
    Holdout scaled features shape: (105215, 14)
  - Predicting with LGBM...
    LGBM predictions shape: (105215,)
    Sample LGBM preds: [0.00014931 0.00013936 0.00017869 0.0001564  0.00013623]
  - Preparing initial LSTM sequence from train residuals...
    Initial sequence shape: (50, 1)
    Initial sequence sample: [-0.70334119 -0.7078674  -0.70798528 -0.7072703  -0.73312855]
  - Generating rolling predictions from LSTM...
    Step 1 | LSTM pred (scaled): -0.716273 | Current seq shape: (1, 50, 1)
    Step 2 | LSTM pred (scaled): -0.715065 | Current seq shape: (1, 50, 1)
    Step 3 | LSTM pred (scaled): -0.719794 | Current seq shape: (1, 50, 1)


In [ ]:

# --- 2. Predictions vs. Actuals Scatter Plot ---
plt.figure(figsize=(10, 10))
plt.scatter(y_holdout, holdout_ensemble_preds, alpha=0.1, s=10)
plt.plot([y_holdout.min(), y_holdout.max()], [y_holdout.min(), y_holdout.max()], 'r--', lw=2, label='Perfect Correlation')
plt.xlabel('True Values (Implied Volatility)', fontsize=12)
plt.ylabel('Ensemble Predictions', fontsize=12)
plt.title(f'Predictions vs. Actuals on Holdout Set\n(Correlation: {holdout_corr:.4f})', fontsize=16)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "predictions_vs_actuals.png")
print("✅ Predictions vs. Actuals plot saved.")

# --- 3. Residuals Plot (Time Series) ---
residuals = y_holdout - holdout_ensemble_preds
plt.figure(figsize=(15, 6))
plt.plot(residuals.values, lw=0.5)
plt.title('Residuals (True - Prediction) on Holdout Set', fontsize=16)
plt.xlabel('Time Step in Holdout Set', fontsize=12)
plt.ylabel('Error', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "residuals_plot.png")
print("✅ Residuals plot saved.")

In [ ]:
# =============================================================================
# 6. VISUALIZATIONS
# =============================================================================
print("\n--- Generating Visualizations ---")

# --- 1. Feature Importance ---
plt.figure(figsize=(10, 8))
lgb.plot_importance(lgbm_model, max_num_features=20, height=0.8, ax=plt.gca())
plt.title('LGBM Model - Top 20 Feature Importances', fontsize=16)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "feature_importance.png")
print("✅ Feature importance plot saved.")

# --- 2. Predictions vs. Actuals Scatter Plot ---
plt.figure(figsize=(10, 10))
plt.scatter(y_holdout, holdout_ensemble_preds, alpha=0.1, s=10)
plt.plot([y_holdout.min(), y_holdout.max()], [y_holdout.min(), y_holdout.max()], 'r--', lw=2, label='Perfect Correlation')
plt.xlabel('True Values (Implied Volatility)', fontsize=12)
plt.ylabel('Ensemble Predictions', fontsize=12)
plt.title(f'Predictions vs. Actuals on Holdout Set\n(Correlation: {holdout_corr:.4f})', fontsize=16)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "predictions_vs_actuals.png")
print("✅ Predictions vs. Actuals plot saved.")

# --- 3. Residuals Plot (Time Series) ---
residuals = y_holdout - holdout_ensemble_preds
plt.figure(figsize=(15, 6))
plt.plot(residuals.values, lw=0.5)
plt.title('Residuals (True - Prediction) on Holdout Set', fontsize=16)
plt.xlabel('Time Step in Holdout Set', fontsize=12)
plt.ylabel('Error', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "residuals_plot.png")
print("✅ Residuals plot saved.")